In [4]:
import keras
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing import image
from keras.engine import Layer
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, merge, concatenate
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard 
from keras.models import Sequential, Model
from keras.layers.core import RepeatVector, Permute
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf

Using TensorFlow backend.


In [5]:
from google.colab import drive
drive.mount('/Mahmoud')

Drive already mounted at /Mahmoud; to attempt to forcibly remount, call drive.mount("/Mahmoud", force_remount=True).


In [0]:
images = []
dir = os.listdir('/Mahmoud/My Drive/newDataset 2')
inc = 1 
for sample in dir:
      if sample[0]!='.':
        img = (img_to_array(load_img('/Mahmoud/My Drive/newDataset 2/'+sample)))/255
        images.append(img)
      if inc==11:
        break
      inc+=1
    

In [7]:
print(len(images))

10


In [8]:
inception = InceptionResNetV2(weights=None, include_top=True)
inception.load_weights('/Mahmoud/My Drive/pretrained model/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels.h5')
inception.graph = tf.get_default_graph()

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
images = np.array(images, dtype=float)


In [0]:
embed_input = Input(shape=(1000,))
#Encoder
encoder_input = Input(shape=(256, 256, 1,))
encoder_output = Conv2D(64, (3,3), activation='relu', padding='same', strides=2)(encoder_input)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)
#Fusion
fusion_output = RepeatVector(32 * 32)(embed_input) 
fusion_output = Reshape(([32, 32, 1000]))(fusion_output)
fusion_output = concatenate([encoder_output, fusion_output], axis=3) 
fusion_output = Conv2D(256, (1, 1), activation='relu', padding='same')(fusion_output)
#Decoder
decoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(fusion_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(64, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(32, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(16, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(2, (3, 3), activation='tanh', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
model = Model(inputs=[encoder_input, embed_input], outputs=decoder_output)

In [12]:
#Create embedding
def create_inception_embedding(grayscaled_rgb):
    grayscaled_rgb_resized = []
    for i in grayscaled_rgb:
        i = resize(i, (299, 299, 3), mode='constant')
        grayscaled_rgb_resized.append(i)
    grayscaled_rgb_resized = np.array(grayscaled_rgb_resized)
    grayscaled_rgb_resized = preprocess_input(grayscaled_rgb_resized)
    with inception.graph.as_default():
        embed = inception.predict(grayscaled_rgb_resized)
    return embed

predictions = []
grayscaled_rgb = gray2rgb(rgb2gray(images))
predictions = create_inception_embedding(grayscaled_rgb)
lab = rgb2lab(images)
X = lab[:,:,:,0]
X = X.reshape(X.shape+(1,))
Y = lab[:,:,:,1:] / 128

print(X.shape)
print(Y.shape)
print(predictions.shape)


(10, 256, 256, 1)
(10, 256, 256, 2)
(10, 1000)


In [0]:
#Train model      
tensorboard = TensorBoard(log_dir="/output")
model.compile(optimizer='adam', loss='mse')
model.fit([X,predictions],Y, callbacks=[tensorboard],validation_split=0.2, epochs=2000)

In [0]:

model.save('my_model_weights.h5')


In [0]:
dir = os.listdir('/Mahmoud/My Drive/newDataset 2')
inc = 1 
for sample in dir:
      if sample[0]!='.':
        print(sample)
      if inc==11:
        break
      inc+=1

In [32]:
from google.colab import files

uploaded = files.upload()

Saving testttt.jpg to testttt.jpg


In [0]:
color_me = []

In [0]:
color_me.append(img_to_array(load_img('testttt.jpg')))

In [0]:
color_me = np.array(color_me, dtype=float)
gray_me = gray2rgb(rgb2gray(1.0/255*color_me))
color_me_embed = create_inception_embedding(gray_me)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))


In [37]:
print(color_me.shape)

(1, 256, 256, 1)


In [38]:
output = model.predict([color_me, color_me_embed])
output = output * 128

# Output colorizations
for i in range(len(output)):
    cur = np.zeros((256, 256, 3))
    cur[:,:,0] = color_me[i][:,:,0]
    cur[:,:,1:] = output[i]
    imsave("result"+".png", lab2rgb(cur))
    #imsave("greyversion"+"png" , )

/usr/local/lib/python3.6/dist-packages/skimage/color/colorconv.py:985: UserWarning: Color data out of range: Z < 0 in 7 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)
/usr/local/lib/python3.6/dist-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


In [0]:
files.download('result.png')